In [13]:
import gym
import gym_chrome_dino
env = gym.make('ChromeDino-v0')
# env = gym.make('ChromeDinoNoBrowser-v0')

In [14]:
from gym_chrome_dino.utils import wrappers
env=wrappers.make_dino(env, timer=True, frame_stack=True)

In [ ]:
env.unwrapped.game

In [ ]:
score = env.unwrapped.game.get_score()
score

In [ ]:
env.unwrapped.game.set_acceleration(True)

In [21]:
import gym
import gym_chrome_dino
import time
import matplotlib.pyplot as plt
import numpy as np

# env = gym.make('ChromeDino-v0')
env = gym.make('ChromeDinoNoBrowser-v0')
# env = make_dino(env, timer=True, frame_stack=True)
done = True
last_time=time.time()
score =0
while True:
    if done:
        env.reset()
        print(score)
        score=0
    print('fps: {0}'.format(1 / (time.time()-last_time)))
    last_time=time.time()
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    score+=reward
    print(done)
    # print(observation)

0
fps: 77.9972849837285
False
fps: 20.163662847995077
False
fps: 93.5936091400009
False
fps: 96.79907685206554
False
fps: 23.301171085087
False
fps: 96.8840432412455
False
fps: 24.835413656711115
False
fps: 23.50816897303538
False
fps: 91.83333698246228
False
fps: 78.8254839315918
False
fps: 96.67413451343751
False
fps: 23.738208161186257
False
fps: 25.248183574821066
False
fps: 24.623274763853257
False
fps: 113.73150031183059
False
fps: 24.611282580887444
False
fps: 96.41191614564178
False
fps: 20.87164915877526
False
fps: 22.799095494868673
False
fps: 25.707480616591585
False
fps: 21.304184846376163
False
fps: 93.85540065788001
False
fps: 21.59885885545674
False
fps: 87.8756337733082
False
fps: 100.60212990501775
False
fps: 103.33852370158668
False
fps: 24.420841800047743
False
fps: 22.15445724457403
False
fps: 21.928478815508804
False
fps: 22.85774076819113
False
fps: 69.42947476452964
False
fps: 91.32743979445195
False
fps: 20.995454818493084
False
fps: 94.47056173701517
False
fps:

KeyboardInterrupt: 

In [22]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D
from tensorflow.keras.optimizers import Adam
height,width,channels=env.observation_space.shape
actions=env.action_space.n


In [ ]:
def build_model(height,width,channels,actions):
    model=Sequential()
    model.add(Conv2D(32,(8,8),strides=(4,4),activation='relu',input_shape=(3,height,width,channels)))
    model.add(Conv2D(64,(4,4),strides=(2,2),activation='relu'))
    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [ ]:
!pip install stable-baselines

In [15]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback
# Check Environment    
from stable_baselines3.common import env_checker

In [16]:
env = gym.make('ChromeDino-v0')

In [43]:
env_checker.check_env(env)

In [17]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
 
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [18]:
CHECKPOINT_DIR = './lasthopeTrainerPPO/'
LOG_DIR = './lasthopeLogsPPO/'

In [19]:
callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)

In [20]:
from stable_baselines3 import DQN,PPO
# from stable_baselines.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [21]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [49]:
import time
time.sleep(2)
model.learn(total_timesteps=100000, callback=callback)

Logging to ./lasthopeLogsPPO/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 74.4     |
|    ep_rew_mean     | 6.34     |
| time/              |          |
|    fps             | 12       |
|    iterations      | 1        |
|    time_elapsed    | 157      |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 71.7       |
|    ep_rew_mean          | 6.07       |
| time/                   |            |
|    fps                  | 9          |
|    iterations           | 2          |
|    time_elapsed         | 440        |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.03669076 |
|    clip_fraction        | 0.177      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.68      |
|    explained_variance   | -0.0401    |
|    learn

In [22]:
model.load("./lasthopeTrainerPPO/best_model_100000.zip",env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [23]:
import time
time.sleep(3)
for episode in range(5): 
    obs = env.reset()
    done = False
    total_reward = 0
    last=time.time()
    while not done:
        # print(f"{1/(time.time()-last)}")
        last=time.time()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(int(action))
        # time.sleep(0.01)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))
    time.sleep(2)

Total Reward for episode 0 is 11.99999999999997
Total Reward for episode 1 is 10.699999999999974
Total Reward for episode 2 is 12.999999999999966
Total Reward for episode 3 is 12.199999999999969
Total Reward for episode 4 is 9.999999999999977
